In [1]:
import os
import NN
import utils
import config as cfg

### Setup Parameters

In [2]:
ETF = 'SPY' # 'SPY', 'DIA', 'QQQ'
NNtype = 'PSN' # 'MLP', 'RNN', 'PSN'
params = cfg.train_parameters[ETF][NNtype]
inputs_lag = cfg.SPYfeatures[NNtype] # SPYfeatures, DIAfeatures, QQQfeatures

### Load Dataframes

In [3]:
traindf = utils.load_file(os.path.join("data", ETF, NNtype, "Train.pkl"))
validdf = utils.load_file(os.path.join("data", ETF, NNtype, "Valid.pkl"))
testdf  = utils.load_file(os.path.join("data", ETF, NNtype, "Test.pkl"))

### Initialize DataLoaders

In [4]:
features_col = []
for i in inputs_lag:
    features_col.append("Return_" + str(i))
target_col   = 'Target'

mu, sigma = None, None

trainloader = utils.DataFrame2DataLoader(traindf, features_col, target_col, batch_size=10, normalize=False)
validloader = utils.DataFrame2DataLoader(validdf, features_col, target_col, batch_size=8, normalize=False)
testloader  = utils.DataFrame2DataLoader(testdf, features_col, target_col, batch_size=8, normalize=False)

### Instanciate Model

In [5]:
model = NN.Model(NNtype)
model.setup(params["input_size"], 
            params["hidden_size"],
            params["output_size"],
            1000,#params["epochs"],
            params["optim"],
            params["lr"],
            params["momentum"]
           )

PSN(
  (fc): Linear(in_features=9, out_features=5, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.4
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)


In [6]:
features, target = next(iter(trainloader))
model.model(features)

tensor([0.3080, 0.3088, 0.3079, 0.3085, 0.3077, 0.3067, 0.3066, 0.3087, 0.3099,
        0.3083], grad_fn=<SigmoidBackward>)

### Training

In [7]:
model.train(trainloader, validloader)

/Users/Chayan/Dev/env/ML/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/Chayan/Dev/env/ML/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/Chayan/Dev/env/ML/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the sa

[00:00] Epoch 1/1000 - Train Loss : 0.0076 / Val Loss : 0.0012
[00:47] Epoch 1000/1000 - Train Loss : 0.0001 / Val Loss : 0.0000
tensor([0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0012, 0.0012, 0.0013, 0.0013,
        0.0013])


TypeError: 'float' object is not iterable

### Evaluation

In [7]:
model.evaluate(testloader)

TypeError: 'float' object is not iterable